In [133]:
import sys
sys.path.append('../utils/')
from parsing import *
import pandas as pd
import re

import warnings
warnings.filterwarnings("ignore")

PATH_INTERMEDIATE = '../../data/intermediate/'
PATH_RAW = '../../data/raw/'
PATH_FINAL = '../../data/final/'

## OZP

In [41]:
insurance_companies = pd.DataFrame()

In [42]:
ozp = pd.read_excel(PATH_RAW+'ic/ozp.xlsx')

In [43]:
ozp.columns = ['year', 'facility_id', 'expertise', 'count'] # rok, identifikacni cislo strediska, odbornost, pocet vykonu
ozp['insurance_company'] = 'OZP'
ozp.info()
insurance_companies = insurance_companies.append(ozp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343327 entries, 0 to 343326
Data columns (total 5 columns):
year                 343327 non-null int64
facility_id          343327 non-null int64
expertise            343325 non-null object
count                343327 non-null int64
insurance_company    343327 non-null object
dtypes: int64(3), object(2)
memory usage: 13.1+ MB


## VZP

In [44]:
vzp = pd.read_excel(PATH_RAW+'ic/vzp.xlsx')

In [45]:
vzp.columns = ['year', 'expertise', 'procedure_code', 'region', 'count'] # rok, odbornost, kod vykonu, region, pocet vykonu
vzp['insurance_company'] = 'VZP'
vzp.info()
insurance_companies = insurance_companies.append(vzp)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455390 entries, 0 to 455389
Data columns (total 6 columns):
year                 455390 non-null int64
expertise            455390 non-null object
procedure_code       455390 non-null int64
region               455390 non-null object
count                455390 non-null int64
insurance_company    455390 non-null object
dtypes: int64(3), object(3)
memory usage: 20.8+ MB


## SKODA

In [46]:
skoda = pd.read_csv(PATH_RAW+'ic/skoda.csv', sep=';', usecols=['odb','sum_vykony','ico','rok'])

In [47]:
skoda.columns = ['expertise', 'count', 'facility_id', 'year'] # rok, odbornost, kod vykonu, region, pocet vykonu
skoda['insurance_company'] = 'SKODA'
skoda.info()
insurance_companies = insurance_companies.append(skoda)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80502 entries, 0 to 80501
Data columns (total 5 columns):
expertise            80491 non-null object
count                80502 non-null int64
facility_id          80435 non-null float64
year                 80502 non-null int64
insurance_company    80502 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 3.1+ MB


In [36]:
insurance_companies.rename(columns={'expertise':'expertise_code'})

insurance_companies.reset_index().to_feather(PATH_INTERMEDIATE+'insurance_companies.ftr')
insurance_companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 879219 entries, 0 to 80501
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   count              879219 non-null  int64  
 1   expertise_code     879206 non-null  object 
 2   facilities_id      423762 non-null  float64
 3   insurance_company  879219 non-null  object 
 4   procedure_code     455390 non-null  float64
 5   region             455390 non-null  object 
 6   year               879219 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 53.7+ MB


## Codes name

In [2]:
ic = pd.read_feather(PATH_INTERMEDIATE+'insurance_companies.ftr').set_index('index')

ic = ic.rename(columns={'expertise':'expertise_code',
                        'facilities_id' : 'ico'})

In [5]:
with open(PATH_RAW+'ic/ODBORN', 'r', encoding="utf-8") as f:
    codes = pd.DataFrame([x.split(';') for x in f.read().splitlines()], columns=['expertise_code', 'expertise_name'])

In [6]:
# set(codes.expertise_code_code.unique()).difference(set(ic.expertise_code.unique()))
unknown_exp = set(ic.expertise_code.unique()).difference(set(codes.expertise_code.unique()))
unknown_count = ic[ic.expertise_code.isin(unknown_exp)]['count'].sum()
total_count = ic['count'].sum()

print(f'Počet nepřiřazených záznamů: {ic[ic.expertise_code.isin(unknown_exp)].shape[0]}/{ic.shape[0]} ({round(100*ic[ic.expertise_code.isin(unknown_exp)].shape[0]/ic.shape[0],2)} %)')
print(f'Počet výkonů z nepřiřazených záznamů: {unknown_count}/{total_count} ({round(100*unknown_count/total_count,2)} %)')

Počet nepřiřazených záznamů: 53154/879219 (6.05 %)
Počet výkonů z nepřiřazených záznamů: 347225075/4953258809 (7.01 %)


In [19]:
ic = ic[~ic.expertise_code.isin(unknown_exp) & (ic['count'] > 75)]
ic_total = pd.merge(ic,codes, on=['expertise_code'])

In [52]:
ic_total.to_csv(PATH_FINAL+'insurances.csv', index=True)

,count,expertise_code,ico,insurance_company,procedure_code,region,year,expertise_name
0,122,001,69909041.0,OZP,NaN,None,2012,praktické lékařství pro dospělé
1,170,001,67011080.0,OZP,NaN,None,2015,praktické lékařství pro dospělé
2,198,001,24235164.0,OZP,NaN,None,2016,praktické lékařství pro dospělé
3,247,001,18231454.0,OZP,NaN,None,2016,praktické lékařství pro dospělé
4,952,001,60700904.0,OZP,NaN,None,2011,praktické lékařství pro dospělé
...,...,...,...,...,...,...,...,...
442375,592,7U5,195201.0,SKODA,NaN,None,2021,dlouhodobá (chronická) ústavní lůžková péče o...
442376,710,7U5,195201.0,SKODA,NaN,None,2017,dlouhodobá (chronická) ústavní lůžková péče o...
442377,1197,7U5,195201.0,SKODA,NaN,None,2019,dlouhodobá (chronická) ústavní lůžková péče o...
442378,984,7U5,195201.0,SKODA,NaN,None,2020,dlouhodobá (chronická) ústavní lůžková péče o...


### Time and expenses

In [16]:
# import tabula
# df = tabula.read_pdf('https://media.vzpstatic.cz/media/Default/dokumenty/ciselniky/vykony_01336.pdf', pages='all')
# dial_df = pd.concat(df)

dial_df = pd.read_csv(PATH_RAW+'ic/vykony_01336.csv', error_bad_lines=False)

dial_df = dial_df.drop_duplicates()
new_header = dial_df.iloc[0]
dial_df = dial_df[1:]
dial_df.columns = new_header
dial_df = dial_df[['KOD', 'ODB', 'NAZ', 'TVY', 'CTN']]
dial_df.columns = ['procedure_code', 'expertise_code', 'procedure_name', 'time_individual', 'time_total']
dial_df = dial_df[~dial_df.expertise_code.isna()].reset_index(drop=True)
dial_df.expertise_code = dial_df.expertise_code.apply(lambda n:f"{n:0>3}")

b'Skipping line 2020: expected 15 fields, saw 16\nSkipping line 2021: expected 15 fields, saw 16\nSkipping line 2022: expected 15 fields, saw 16\nSkipping line 2023: expected 15 fields, saw 16\nSkipping line 2024: expected 15 fields, saw 16\nSkipping line 2025: expected 15 fields, saw 16\nSkipping line 2026: expected 15 fields, saw 16\nSkipping line 2027: expected 15 fields, saw 16\nSkipping line 2028: expected 15 fields, saw 16\nSkipping line 2029: expected 15 fields, saw 16\nSkipping line 2030: expected 15 fields, saw 16\nSkipping line 2031: expected 15 fields, saw 16\nSkipping line 2032: expected 15 fields, saw 16\nSkipping line 2033: expected 15 fields, saw 16\nSkipping line 2034: expected 15 fields, saw 16\nSkipping line 2035: expected 15 fields, saw 16\nSkipping line 2036: expected 15 fields, saw 16\nSkipping line 2037: expected 15 fields, saw 16\nSkipping line 2038: expected 15 fields, saw 16\nSkipping line 2039: expected 15 fields, saw 16\nSkipping line 2040: expected 15 fields

In [18]:
# https://www.vzp.cz/poskytovatele/ciselniky/zdravotni-vykony
# https://szv.mzcr.cz/Vykon?page=0

dial_df2 = pd.DataFrame()
for i in range(4):
    x = pd.read_html(f'https://szv.mzcr.cz/Ciselnik/Odbornost?page={i}') # 10.11.2022
    dial_df2 = dial_df2.append(x[0])

dial_df2 = dial_df2.drop_duplicates()
dial_df2 = dial_df2[['Kód', 'Název', 'Sazba režie']]
dial_df2.columns = ['expertise_code', 'expertise_name', 'tariff']
dial_df2.expertise_code = dial_df2.expertise_code.apply(lambda n:f"{n:0>3}")

In [101]:
dial_df[~dial_df.expertise_code.isin(dial_dfs.expertise_code.unique())].expertise_code.unique()

array(['000', '999', '921', '911', '925', '916', '927', '913', '904',
       '902', '917', '910', '355', '914', '901', '931', '370', '919',
       '903', '905', '926', '929', '920', '935', '961', '922'],
      dtype=object)

In [100]:
dial_dfs = pd.merge(dial_df, dial_df2, on='expertise_code')
dial_dfs['expertise_name'] = dial_dfs['expertise_name'].apply(lambda x: x.split('-')[0].strip())
dial_dfs['time_individual'] = dial_dfs['time_individual'].apply(pd.to_numeric)
dial_dfs['time_extra'] = dial_dfs['time_individual'] + 5
dial_dfs

,procedure_code,expertise_code,procedure_name,time_individual,time_total,expertise_name,tariff,time_extra
0,130,304,NaN,480.0,1920,neonatologie,351,485.0
1,132,304,VYŠETŘENÍ DÍTĚTE S VÝZNAMNÝM PERINATÁLNÍM RIZIKEM,70.0,70,neonatologie,351,75.0
2,34007,304,EDUKACE LAKTACE,45.0,45,neonatologie,351,50.0
3,34045,304,CELOTĚLOVÁ HYPOTERMIE NOVOROZENCE,1440.0,720,neonatologie,351,1445.0
4,34046,304,SCREENING VROZENÉ KATARAKTY,15.0,15,neonatologie,351,20.0
...,...,...,...,...,...,...,...,...
5000,99147,812,JINÁ LÉČIVA A METABOLITY LÉČIV JEDNOTLIVĚ,10.0,10,laboratoř farmakologie a toxikologie léčiv,351,15.0
5001,99149,812,LITHIUM,2.0,2,laboratoř farmakologie a toxikologie léčiv,351,7.0
5002,99151,812,PSYCHOTROPNÍ LÁTKY JEDNOTLIVĚ,5.0,5,laboratoř farmakologie a toxikologie léčiv,351,10.0
5003,99153,812,IMUNOSUPRESIVA JEDNOTLIVĚ,5.0,5,laboratoř farmakologie a toxikologie léčiv,351,10.0


In [118]:
def expertise_to_specialty(name):
    return {
        'nemocniční lékárenství' : 'nemocniční lékárenství',
        'angiologie' : 'angiologie',
        'gastroenterologie' : 'gastroenterologie',
        'geriatrie' : 'geriatrie',
        'nefrologie' : 'nefrologie',
        'revmatologie' : 'revmatologie',
        'klinická osteologie' : 'klinická osteologie',
        'medicína dlouhodobé péče' : 'medicína dlouhodobé péče',
        'infekční lékařství' : 'infekční lékařství',
        'tělovýchovné lékařství' : 'tělovýchovné lékařství',
        'pneumologie a ftizeologie' : 'pneumologie a ftizeologie',
        'klinická farmakologie' : 'klinická farmakologie',
        'alergologie a klinická imunologie' : 'alergologie a klinická imunologie',
        'lékařská genetika' : 'lékařská genetika',
        'neurologie' : 'neurologie',
        'dětská neurologie' : 'dětská neurologie',
        'dětská kardiologie' : 'dětská kardiologie',
        'psychiatrie' : 'psychiatrie',
        'návykové nemoci' : 'návykové nemoci',
        'sexuologie' : 'sexuologie',
        'centrum duševního zdraví' : 'centrum duševního zdraví',
        'Centrum duševního zdraví pro děti a dorost' : 'Centrum duševního zdraví pro děti a dorost',
        'pracovní lékařství' : 'pracovní lékařství',
        'klinická onkologie' : 'klinická onkologie',
        'radiační onkologie' : 'radiační onkologie',
        'dermatovenerologie' : 'dermatovenerologie',
        'dětská dermatologie' : 'dětská dermatologie',
        'korektivní dermatologie' : 'korektivní dermatologie',
        'neurochirurgie' : 'neurochirurgie',
        'hrudní chirurgie' : 'hrudní chirurgie',
        'algeziologie' : 'algeziologie',
        'foniatrie' : 'foniatrie',
        'dětská otorinolaryngologie' : 'dětská otorinolaryngologie',
        'urgentní medicína' : 'urgentní medicína',
        'paliativní medicína' : 'paliativní medicína',
        'hyperbarická a letecká medicína' : 'hyperbarická medicína a oxygenoterapie',
        'přeprava pacientů neodkladné péče' : 'přeprava pacientů neodkladné péče',
        'lékařská mikrobiologie' : 'lékařská mikrobiologie',
        'mamografický screening' : 'radiologie a zobrazovací metody',
        'patologická anatomie' : 'patologie',
        'soudní lékařství' : 'soudní lékařství',
        'radiologie a zobrazovací metody' : 'radiologie a zobrazovací metody',
        'transfuzní lékařství' : 'hematologie a transfuzní lékařství',
        'dětské lékařství' : 'pediatrie',
        'dorostové lékařství' : 'pediatrie',
        'orální a maxilofaciální chirurgie' : 'maxilofaciální chirurgie',
        'dětská a dorostová psychiatrie' : 'dětská psychiatrie',
        'klinická farmacie' : 'klinická farmakologie',
        'klinická stomatologie' : 'stomatologie',
        'ortodoncie' : 'stomatologie',
        'pracoviště hemodialýzy' : 'nefrologie',
        'rehabilitační a fyzikální medicína' : '',
        'hematologie' : 'hematologie a transfuzní lékařství',
        'diabetologie' : 'endokrinologie a diabetologie',
        'endokrinologie' : 'endokrinologie a diabetologie',
        'neonatologie':'neonatologie',
        'kardiochirurgie':'kardiochirurgie',
        'chirurgie':'chirurgie',
        'cévní chirurgie':'cévní chirurgie',
        'gynekologie a porodnictví':'gynekologie a porodnictví',
        'dětská chirurgie':'dětská chirurgie',
        'úrazová chirurgie (traumatologie)':'ortopedie a traumatologie pohybového ústrojí',
        'plastická chirurgie':'plastická chirurgie',
        'popáleninová medicína':'popáleninová medicína',
        'dětská gynekologie':'gynekologie dětí a dospívajících',
        'ortopedie':'ortopedie a traumatologie pohybového ústrojí',
        'otorinolaryngologie':'otorinolaryngologie a chirurgie hlavy a krku',
        'urologie':'urologie',
        'anesteziologie a intenzivní medicína':'anesteziologie a intenzivní medicína',
        'kardiologie':'kardiologie',
        'vnitřní lékařství - interna':'vnitřní lékařství',
        'vnitřní lékařství':'vnitřní lékařství',
        'nukleární medicína':'nukleární medicína',
        'ortopedická protetika' : 'ortopedie a traumatologie pohybového ústrojí',
        'oftalmologie' : 'oftalmologie',
        'dětská urologie' : 'dětská urologie',
        'klinická biochemie' : 'klinická biochemie',
        'laboratoř alergologická a imunologická' : 'laboratoř alergologická a imunologická',
        'laboratoř lékařské genetiky' : 'laboratoř lékařské genetiky',
        'laboratoř klinické cytologie' : 'laboratoř klinické cytologie',
        'laboratoř hematologická' : 'laboratoř hematologická',
        'laboratoř farmakologie a toxikologie léčiv' : 'laboratoř farmakologie a toxikologie léčiv',
        'laboratoř toxikologická' : 'laboratoř toxikologická',
        'laboratoř nukleární medicíny' : 'laboratoř nukleární medicíny',
        'laboratoř pro univerzální výkony' : 'laboratoř pro univerzální výkony',
        'laboratoř patologie' : 'laboratoř patologie',
        'všeobecné praktické lékařství' : 'všeobecné praktické lékařství',
        'praktické lékařství pro děti a dorost' : 'pediatrie',
        'gerontopsychiatrie': 'psychiatrie',
        'audiologie': 'foniatrie',
        'přenosné nemoce': 'venerologie', 

        'hemodialýza' : 'hemodialýza',
        'lékařská služba první pomoci' : 'lékařská služba první pomoci',
        'laboratoř provádějící screening karcinomu děložního hrdla' : 'laboratoř provádějící screening karcinomu děložního hrdla',
        'stomatologická LSPP' : 'stomatologická LSPP',
        'lékařská parazitologie' : 'lékařská parazitologie',
        'lékařská virologie' : 'lékařská virologie',
        'laboratoř mykologická' : 'laboratoř mykologická',
        'lékařská bakteriologie':'lékařská bakteriologie',
        }.get(name, None)

dial_dfs['medical_specialty'] = dial_dfs['expertise_name'].apply(expertise_to_specialty)


In [119]:
time_expertise_mean = dial_dfs[['expertise_code', 'time_individual']].groupby('expertise_code').mean().rename(columns={'time_individual':'time_expertise_mean'}).reset_index()
dial_df_total = pd.merge(dial_dfs, time_expertise_mean)
dial_df_total.procedure_code = dial_df_total.procedure_code.apply(pd.to_numeric)
dial_df_total = dial_df_total[['procedure_code', 'expertise_code', 'time_individual', 'medical_specialty', 'time_expertise_mean']]

In [120]:
ic = pd.read_csv(PATH_FINAL+'insurances.csv', index_col=0)
ic.expertise_code = ic.expertise_code.apply(lambda n:f"{n:0>3}")

In [121]:
exp_proc = pd.merge(dial_df_total,
                    ic[~ic['procedure_code'].isna()],
                    on=['expertise_code', 'procedure_code'],
                    how='right')

exp = pd.merge(dial_df_total[['expertise_code', 'medical_specialty', 'time_expertise_mean']].drop_duplicates().reset_index(drop=True),
                    ic[ic['procedure_code'].isna()],
                    on=['expertise_code'],
                    how='right')
                    
assert exp_proc.shape[0] + exp.shape[0] == ic.shape[0]

In [122]:
ic_extra = pd.concat([exp_proc, exp]).reset_index(drop=True)
tmp = ic_extra[~ic_extra['medical_specialty'].isna()][['expertise_code', 'medical_specialty']].drop_duplicates()
exp_ms_dict = dict(zip(tmp.expertise_code, tmp.medical_specialty))
ic_extra.loc[ic_extra.medical_specialty.isna(), 'medical_specialty'] = ic_extra[ic_extra.medical_specialty.isna()]['expertise_code'].apply(lambda x: exp_ms_dict.get(x, None))
ic_extra['expertise_name'] = ic_extra['expertise_name'].apply(lambda x: x.strip())
ic_extra.loc[ic_extra.medical_specialty.isna(), 'medical_specialty'] = ic_extra[ic_extra.medical_specialty.isna()]['expertise_name'].apply(expertise_to_specialty)

In [132]:
ic_extra.to_csv(PATH_FINAL+'insurances_extra.csv')

## NRPZS

In [16]:
# nrpzs = pd.read_csv('https://opendata.mzcr.cz/data/nrpzs/narodni-registr-poskytovatelu-zdravotnich-sluzeb.csv') # 25. 9. 2022
nrpzs = pd.read_csv(PATH_RAW+'nrpzs.csv')

nrpzs.columns = [re.sub(r"(?<=[a-z])([A-Z])", r"_\1", x).lower() for x in nrpzs.columns]

not_required = ['Lékárna',
                'Samostatné zařízení fyzioterapeuta',
                'Zdravotnická zachranná služba',
                'Výdejna zdravotnických prostředků']
nrpzs = nrpzs[~nrpzs.druh_zarizeni.isin(not_required)].reset_index(drop=True)

nrpzs = nrpzs[[x for x in nrpzs.columns if not 'poskytovatel_' in x]]


nrpzs.loc[nrpzs.okres == 'území Hlavního města Prahy', 'okres'] = 'Hlavní město Praha'
nrpzs.loc[nrpzs.okres.isna() & (nrpzs.kraj == 'Hlavní město Praha'), 'okres'] = 'Hlavní město Praha'
nrpzs.loc[nrpzs.kraj.isna() & (nrpzs.okres == 'Hlavní město Praha'), 'kraj'] = 'Hlavní město Praha'
nrpzs.loc[nrpzs.kraj.isna() & (nrpzs.okres == 'Náchod'), 'kraj'] = 'Královéhradecký kraj'
nrpzs.loc[nrpzs.kraj.isna() & (nrpzs.okres == 'Pelhřimov'), 'kraj'] = 'Kraj Vysočina'

nrpzs.to_csv(PATH_FINAL+'nrpzs.csv')